In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mnist
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from skimage.util import random_noise
from skimage.filters import threshold_local
import os
from PIL import Image
import cv2

In [ ]:
X_train, y_train = mnist.train_images(), mnist.train_labels()
X_test, y_test = mnist.test_images(), mnist.test_labels()

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

## Zaj hozzáadása az adathalmazhoz

In [ ]:
noise_level = 0.1

X_train_noisy = np.array([random_noise(image, mode='gaussian', var=noise_level).flatten() for image in X_train])
X_test_noisy = np.array([random_noise(image, mode='gaussian', var=noise_level).flatten() for image in X_test])

classes = np.unique(y_test)
plt.figure(figsize=(12, 12))
for i, cls in enumerate(classes):
    plt.subplot(5, 5, i + 1)
    idx = np.where(y_test == cls)[0][0]
    plt.imshow(X_test_noisy[idx].reshape(28, 28), cmap='gray')
    plt.title(f'Osztály: {cls}')
    plt.axis('off')
plt.show()

In [ ]:
clf = MLPClassifier(solver='adam', activation='tanh', hidden_layer_sizes=(64, 64),
                    max_iter=1, learning_rate='adaptive')

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for i in range(85):
    clf.partial_fit(X_train_noisy, y_train, classes=np.unique(y_train))

    y_train_pred = clf.predict(X_train_noisy)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)

    train_loss = clf.loss_
    train_losses.append(train_loss)

    y_test_pred = clf.predict(X_test_noisy)
    val_accuracy = accuracy_score(y_test, y_test_pred)
    val_accuracies.append(val_accuracy)

    val_loss = clf.loss_
    val_losses.append(val_loss)

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Tanító adathalmaz')
plt.plot(val_accuracies, label='Validációs adathalmaz')
plt.xlabel('Iterációk')
plt.ylabel('Pontosság')
plt.title('Pontosság változása az iterációk számával')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_losses)
plt.plot(val_losses)
plt.xlabel('Iterációk')
plt.ylabel('Hiba')
plt.title('Hiba változása az iterációk számával')

plt.tight_layout()
plt.show()

In [ ]:
def image_to_bin(img):
    data = list(img.getdata())
    for i in range(len(data)):
        data[i] = 255 - data[i]
    data = np.array(data)/256.0
    return data

# Gimpben rajzolt számok

In [ ]:
image_folder = "drawn_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Kézzel írott számok

In [ ]:
image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Zajosított számjegyek

In [ ]:
def add_noise(image, mean=10, std_dev=15):
    img_array = np.array(image)
    noise = np.random.normal(mean, std_dev, img_array.shape).astype(np.uint8)
    noisy_image = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image, mode='L')  # Az 'L' mód jelzi a szürkeárnyalatos képet

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        noisy_image = add_noise(image)
        binimg = image_to_bin(noisy_image)
        p = clf.predict([binimg.flatten()])
        images.append(noisy_image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


# Küszöbölés invertálással

In [ ]:
def local_threshold_mean(image, kernel_size):
    blurred_image = cv2.blur(image, (kernel_size, kernel_size))
    convolved_image = cv2.filter2D(image, -1, np.ones((kernel_size, kernel_size), dtype=np.float32) / (kernel_size * kernel_size))
    thresholded_image = np.where(image > convolved_image, 255, 0).astype(np.uint8)
    return thresholded_image

def invert_colors(img):
    return cv2.bitwise_not(img)

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) 
        binimg = local_threshold_mean(image, kernel_size=25)
        inverted_img = invert_colors(binimg)
        p = clf.predict([inverted_img.flatten()])
        images.append(inverted_img)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Kontúrkövetéssel

In [ ]:
def preprocess_image(image):
    blurred_image = cv2.GaussianBlur(image, (1,1), 0)
    _, binary_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_img = image.copy()
    cv2.drawContours(contour_img, contours, -1, (255,0, 255), -1)
    return contour_img

image_folder = "handwritten_digits"
images = []
predictions = []

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        processed_image = preprocess_image(image)

        image_with_contours = contours(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB))
        image_with_contours_gray = cv2.cvtColor(image_with_contours, cv2.COLOR_RGB2GRAY)

        p = clf.predict([image_with_contours_gray.flatten()])
        images.append(image_with_contours)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Észrevételek

- Kisebb modell pontosság
- Hosszabb tanulási idő
- Jobb becslés a feldolgozatlan képeken
- Feldolgozott képek felismerése hibásabb